## Подготовка датасета

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
from IPython.display import Markdown, HTML
import re

### Объявление необходимых данных

Здесь мы объявляем данные об имени аттрибутов и их значений, нужных для парсинга, хедеры, и т.д.

In [2]:
label_values = np.array(['author', 'translator', 'editor', 'publisher', 'genre', 'series'],
                        dtype=str)
class_values = np.array(['articul', 'isbn', 'pages2', 'weight', 'buying-price-val-number'],
                       dtype=str)
parse_attrs = {'data-event-label' : label_values, 'class' : class_values}


headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0'
      }

cols = np.array(['Автор', 'Переводчик', 'Редактор', 'Издательство', 'Жанр', 
                 'Артикул', 'ISBN', 'Количество страниц', 'Масса', 'Цена'])


info = {}

### Функции, необходимые для парсинга

In [162]:
def parse_one_book(url, df):
    """
    Функция для парсинга страницы по одной книге
    
    url: ссылка на страницу книги
    df: pandas DataFrame для добавления в него данных о книге. 
    
    returns: Новый датафрейм
    """
    
    response = requests.get(url, headers=headers)    
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('div', class_='product-description')
    
    info = {}
    for i in items:
        for attr in parse_attrs:
            for value in parse_attrs[attr]:  
                info[value] = [v.text for v in i.find_all(attrs={attr : value})]
                
    items = soup.find_all('div', attrs={'id' : 'product-voting'})
    for i in items:
        info['rate'] = i.find_all('div', attrs={'id' : 'rate'})[0].text
        info['count-marks-label'] = i.find_all('div', attrs={'id' : 'count-marks-label'})[0].text
    
    
    items = soup.find_all('div', class_='prodtitle')
    info['name'] = (items[0].find_all('h1')[0].text).split(':')[-1].strip()

    print(info)
    df = df.append(info, ignore_index=True)
    
    return df

In [135]:
def parse_books_from_page(url, df, filename='dataset.csv'):
    """
    Функция для парсинга всех книг с одной страницы
    
    url: ссылка на страницу с книгами
    df: pandas DataFrame для добавления в него данных о книгах.
    
    returns: Новый датафрейм
    """
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('a', class_='product-title-link', href=True)
    
    for i in items:
        link = f'https://www.labirint.ru{i["href"]}'
        print(link)
        df = parse_one_book(link, df)
        time.sleep(np.random.rand() * 0.5)
        
    df.to_csv(filename)
    return df

In [263]:
def parse_books_from_all_pages(main_url, df, filename='dataset.csv'):
    """
    Функция для парсинга информации о книгах со всех страниц одного подраздела
    
    main_url: префикс ссылки на страницу раздела
    df: датафрейм для подгрузки полученных данных
    filename: имя файла в которые идет попутное сохранение
    
    returns: датафрейм с полученными данными
    """
    
    urls = [f'{main_url}?page={i}' for i in range(1, 6)]
    np.random.shuffle(urls)
    for url in urls:
        df = parse_books_from_page(url, df, filename)
        time.sleep(np.random.rand())
    return df

In [264]:
def parse_books_from_all_urls(urls, df, filename='dataset.csv'):
    """
    Функция для парсинга информации о книгах со всех ссылок из списка
    
    urls: список ссылок
    df: датафрейм для подгрузки полученных данных
    filename: имя файла, в который идет попутное сохранение
    
    returns: датафрейм с полученными данными
    """
    
    np.random.shuffle(urls)
    for url in urls:
        df = parse_books_from_all_pages(f'https://www.labirint.ru{url}', df, filename)
        print('OKEY URL')
        time.sleep(np.random.rand() * 1.8)
        
    return df

In [261]:
def find_urls(main_url):
    """
    Функция для нахождения ссылок на подразделы
    
    main_url: ссылка на страницу с разделом, где ссылки на подразделы
    
    returns: список ссылок
    """
    
    response = requests.get(main_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('div', 'subgenres')
    
    result = []
    for i in items:
        for href in i.find_all(attrs={'rel':'nofollow'}):
            result.append(href['href'])
            
    return result

In [262]:
def find_all_urls(main_url):
    """
    Функция для нахождения всех ссылок на разделы
    
    main_url: ссылка на главную страницу сайта, где находятся ссылки на все разделы
    
    returns: список ссылок
    """
    
    urls = find_urls(main_url)
    
    result = []
    for url in urls:
        result.extend(find_urls(f'https://www.labirint.ru{url}'))
        print('OKEY PAGE')
        time.sleep(np.random.rand() * 1.5)
        
    return result

In [293]:
def edit_dataframe(df):
    """
    Функция для редактирования значений из датафрейма
    Приводит данные в приемлимый вид
    
    df: датафрейм с данными 
    
    returns: датафрейм с отредактированными данными
    """
    
    result = df.copy()
    for index, row in result.iterrows():
        for column in result.columns:
            if (isinstance(result[column][index], list)):
                result[column][index] = ', '.join(result[column][index])
                print(result[column][index])
            
            if isinstance(result[column][index], str):
                for sym in ['[', ']', '\'']:
                    result[column][index] = result[column][index].replace(sym, '')
        
    for index, row in result.iterrows():
        new_articul = ''.join(re.findall(r'\d+', row['articul']))
        new_pages = ''.join(re.findall(r'\d+', row['pages2']))
        new_weight = ''.join(re.findall(r'\d+', row['weight']))
        new_count = ''.join(re.findall(r'\d+', row['count-marks-label']))       
        new_isbn = '-'.join(re.findall(r'\d+', row['isbn']))
        
        result['articul'][index] = new_articul
        result['pages2'][index] = new_pages
        result['weight'][index] = new_weight
        result['count-marks-label'][index] = new_count
        result['isbn'] = new_isbn
            
    return result

### Парсинг в датафрейм

In [6]:
df = pd.DataFrame(columns=list(label_values) + list(class_values) + 
                  ['rate', 'count-marks-label'])

In [299]:
new_df.to_csv('full_dataset3.csv', index=False)

In [300]:
data = pd.read_csv('full_dataset3.csv')

In [301]:
html = data.to_html()
display(HTML(html))

,Unnamed: 0,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,4,Пётр I,"Новичкова Елена, Ратина Анна, Бунтман Екатерина",NaN,Новичкова Елена,Лабиринт,NaN,Детская художественная литература,507700,978-5-17-134106-0,2820.0,1200.0,3680,7.80,327
1,5,Unity и C#. Геймдев от идеи до реализации,Бонд Джереми Гибсон,Киселев А.,Тульцева К.,Питер,NaN,Для профессионалов,686355,978-5-17-134106-0,928.0,1190.0,2975,9.70,10
2,6,Грокаем алгоритмы. Иллюстрированное пособие для программистов и любопытствующих,Бхаргава Адитья,Матвеев Е.,Римицан Н.,Питер,NaN,Библиотека программиста,571060,978-5-17-134106-0,288.0,376.0,1024,8.89,44
3,7,Компьютерные сети,"Таненбаум Эндрю, Уэзеролл Дэвид",Гребеньков А.,Сергиенко Ю.,Питер,NaN,Классика computer science,310025,978-5-17-134106-0,960.0,1344.0,2312,9.47,40
4,8,Интерфейс. Основы проектирования взаимодействия,"Купер Алан, Носсел Кристофер, Кронин Дэвид, Рейман Роберт",Матвеев Е.,Римицан Н.,Питер,NaN,Для профессионалов,521205,978-5-17-134106-0,720.0,914.0,2197,9.15,13
5,9,Идеальный программист. Как стать профессионалом разработки ПО,Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,643363,978-5-17-134106-0,224.0,300.0,845,10.00,10
6,10,Современные операционные системы,"Таненбаум Эндрю, Бос Херберт","Леонтьева А. В., Малышева М., Вильчинский Н.",Сергиенко Ю.,Питер,NaN,Классика computer science,485463,978-5-17-134106-0,1120.0,1550.0,2178,9.48,21
7,11,"“Непрактичный” Python. Занимательные проекты для тех, кто хочет поумнеть",Воган Ли,Логунов Андрей,NaN,BHV,NaN,NaN,785345,978-5-17-134106-0,464.0,606.0,1210,0.00,0
8,12,Самоучитель КОМПАС-3D V19,Герасимов Анатолий Александрович,NaN,NaN,BHV,NaN,Самоучитель,785346,978-5-17-134106-0,624.0,805.0,1361,0.00,0
9,13,Разработка приложений на Swift 5.1 и SwiftUI с нуля,Казанский Александр Анатольевич,NaN,NaN,BHV,NaN,С нуля,785347,978-5-17-134106-0,384.0,490.0,1064,0.00,0


,Unnamed: 0,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
4,4,Пётр I,"Новичкова Елена, Ратина Анна, Бунтман Екатерина",,Новичкова Елена,Лабиринт,,Детская художественная литература,507700,978-5-17-134106-0,2820,1200,3680,7.80,327
5,5,Unity и C#. Геймдев от идеи до реализации,Бонд Джереми Гибсон,Киселев А.,Тульцева К.,Питер,,Для профессионалов,686355,978-5-17-134106-0,928,1190,2975,9.70,10
6,6,Грокаем алгоритмы. Иллюстрированное пособие дл...,Бхаргава Адитья,Матвеев Е.,Римицан Н.,Питер,,Библиотека программиста,571060,978-5-17-134106-0,288,376,1024,8.89,44
7,7,Компьютерные сети,"Таненбаум Эндрю, Уэзеролл Дэвид",Гребеньков А.,Сергиенко Ю.,Питер,,Классика computer science,310025,978-5-17-134106-0,960,1344,2312,9.47,40
8,8,Интерфейс. Основы проектирования взаимодействия,"Купер Алан, Носсел Кристофер, Кронин Дэвид, Ре...",Матвеев Е.,Римицан Н.,Питер,,Для профессионалов,521205,978-5-17-134106-0,720,914,2197,9.15,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,263,Численные методы и программирование. Учебное п...,Слабнов Виктор Дмитриевич,,,Лань,,Компьютеры и программное обеспечение,786067,978-5-17-134106-0,460,714,3975,0.00,0
264,264,Web-программирование на JavaScript. Учебное по...,Диков Андрей Валентинович,,,Лань,,Компьютеры и программное обеспечение,786072,978-5-17-134106-0,168,278,1492,0.00,0
265,265,Технология разработки программного обеспечения...,Зубкова Татьяна Михайловна,,,Лань,,Компьютеры и программное обеспечение,786076,978-5-17-134106-0,252,450,2386,0.00,0
266,266,Основы построения инфокоммуникационных сетей и...,"Пуговкин Алексей Викторович, Покаместов Дмитри...",,,Лань,,Компьютеры и программное обеспечение,786078,978-5-17-134106-0,176,354,1889,1.00,1
